# Scrape Website (Javascript sites)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from colorama import Fore, Back, Style
import csv
import os
import pandas as pd
import re
import requests
import sys
import time
import urllib.parse
from IPython.display import clear_output
from przona import *

dummy = csv.field_size_limit(sys.maxsize)

In [ ]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

In [ ]:
def store_content(local_file_name, page_source):
    local_file_name_parts = local_file_name.split("/")
    for i in range(1, len(local_file_name_parts)):
        if not os.path.isdir("/".join(local_file_name_parts[0: i])):
            os.mkdir("/".join(local_file_name_parts[0: i]))
    local_file = open(local_file_name, "w")
    print(page_source, file=local_file)
    local_file.close()

In [ ]:
def make_local_file_name(website, remote_file_name):
    base_dir_name = f"../data/{'.'.join(website.split('.')[-2:])}"
    remote_file_parts = remote_file_name.split("/")
    if re.search(r'\.html$', remote_file_name):
        dir_name = "/".join(remote_file_parts[:-1])
        file_name = remote_file_parts[-1]
    else:
        dir_name = remote_file_name
        file_name = "index.html"
    dir_name = re.sub("^/*", "", dir_name)
    local_dir_name = os.path.join(base_dir_name, dir_name)
    local_file_name = os.path.join(local_dir_name, file_name)
    return local_file_name

In [ ]:
def get_page_links(page_source, patterns=[]):
    page_links = []
    for a in BeautifulSoup(page_source, "html.parser").select('a'):
        try:
            href = a.get("href")
            for pattern in patterns:
                if re.search(pattern, href):
                    page_links.append(href)
                    break
        except TypeError:
            pass
    return(page_links)

In [ ]:
def get_web_page(website, remote_file_name):
    url = website + remote_file_name
    squeal(f"fetching {url}")
    assert re.search(r"^https://", url), f"get_web_page: url has unexpected format: {url}"
    time.sleep(1)
    try:
        driver = webdriver.Firefox()
        driver.get("https://www.ggzstandaarden.nl")
        time.sleep(5)
        page_source = driver.page_source
        driver.quit()
    except Exception as e:
        print(f"web scraper error for page {url}: {str(e)}")
        page_source = ""
    return page_source

In [ ]:
def get_web_pages(website, remote_file_name, patterns=[], processed_urls=[]):
    if remote_file_name != "" and not re.search("/.*/.*/", remote_file_name):
        remote_file_name += "/introductie"
    if remote_file_name in processed_urls or re.search(r'\.pdf$', remote_file_name) or re.search(r'\?', remote_file_name):
        return
    local_file_name = make_local_file_name(website, remote_file_name)
    if os.path.isfile(local_file_name) and not re.search("index.html$", local_file_name):
        processed_urls[remote_file_name] = ""
        return
    processed_urls[remote_file_name] = get_web_page(website, remote_file_name)
    store_content(local_file_name, processed_urls[remote_file_name])
    page_links = get_page_links(processed_urls[remote_file_name], patterns)
    for page_link in sorted(page_links):
        get_web_pages(website, page_link, patterns, processed_urls)
        break
    return

In [ ]:
url = "https://www.ggzstandaarden.nl"
patterns = ["^/generieke-modules/", "^/richtlijnen/", "^/zorgstandaarden/",]

processed_urls = {}
get_web_pages(url, "", patterns, processed_urls)
len(processed_urls)